In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# Install necessary packages
!pip install -q kaggle
!pip install -q tensorflow tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.7 MB/s eta 0:00:00


In [ ]:
# Upload Kaggle API credentials
from google.colab import files
files.upload()  # Upload your kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aasthaadhikari012","key":"9d07d51bc7bdbf83ee16d41d362e0ed5"}'}

In [ ]:
# Configure Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign


Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
 98% 601M/612M [00:03<00:00, 212MB/s]
100% 612M/612M [00:03<00:00, 199MB/s]


In [ ]:
# Unzip the dataset
!unzip -q gtsrb-german-traffic-sign.zip -d traffic_sign_data

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [ ]:
# Path to dataset
data_path = "traffic_sign_data/Train"

In [ ]:
# Load dataset
def load_data(data_path):
    images, labels = [], []
    for class_id in os.listdir(data_path):
        class_dir = os.path.join(data_path, class_id)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (32, 32))  # Resize to standard size
            images.append(img)
            labels.append(int(class_id))
    return np.array(images), np.array(labels)

In [ ]:
# Load and preprocess
X, y = load_data(data_path)
X = X / 255.0  # Normalize
y = to_categorical(y, num_classes=len(np.unique(y)))

In [ ]:
# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (31367, 32, 32, 3)
Validation data shape: (7842, 32, 32, 3)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten
from tensorflow.keras.models import Model


In [ ]:
# Base Model
input_layer = Input(shape=(32, 32, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_layer)


In [ ]:
# Custom Head for Traffic Sign Classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
output_layer = Dense(len(np.unique(y)), activation="softmax")(x)


In [ ]:
# Correct the final output layer for 43 classes
output_layer = Dense(43, activation="softmax")(x)


In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Compile Model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

Epoch 1/10
981/981 [==============================] - 209s 197ms/step - loss: 2.3369 - accuracy: 0.3295 - val_loss: 3.8585 - val_accuracy: 0.0403
Epoch 2/10
981/981 [==============================] - 191s 195ms/step - loss: 1.1587 - accuracy: 0.6297 - val_loss: 1.5925 - val_accuracy: 0.5385
Epoch 3/10
981/981 [==============================] - 191s 195ms/step - loss: 0.9901 - accuracy: 0.6911 - val_loss: 0.4757 - val_accuracy: 0.8483
Epoch 4/10
981/981 [==============================] - 191s 194ms/step - loss: 0.3288 - accuracy: 0.8972 - val_loss: 0.3679 - val_accuracy: 0.8860
Epoch 5/10
981/981 [==============================] - 190s 194ms/step - loss: 0.2272 - accuracy: 0.9311 - val_loss: 0.2524 - val_accuracy: 0.9204
Epoch 6/10
981/981 [==============================] - 192s 196ms/step - loss: 0.2010 - accuracy: 0.9414 - val_loss: 0.2602 - val_accuracy: 0.9248
Epoch 7/10
981/981 [==============================] - 190s 194ms/step - loss: 0.1796 - accuracy: 0.9467 - val_loss: 0.3870 -

In [ ]:
# Save the model
model.save("traffic_sign_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load saved model
model = tf.keras.models.load_model("traffic_sign_model.h5")

In [ ]:
# Simulate a single test
def simulate_real_time(image):
    image_resized = cv2.resize(image, (32, 32)) / 255.0
    prediction = model.predict(np.expand_dims(image_resized, axis=0))
    class_id = np.argmax(prediction)
    return class_id

In [ ]:
# Simulated Camera Input
test_image = cv2.imread("/content/traffic_sign_data/Test/00000.png")
predicted_class = simulate_real_time(test_image)
print(f"Detected Class ID: {predicted_class}")

1/1 [==============================] - 1s 909ms/step
Detected Class ID: 16


In [ ]:
# Evaluate detection accuracy
results = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {results[1] * 100:.2f}%")

Validation Accuracy: 96.38%


In [ ]:
# Add evaluation for latency (if applicable)
import time


In [ ]:
start_time = time.time()
simulate_real_time(test_image)
end_time = time.time()

1/1 [==============================] - 0s 47ms/step


In [ ]:
print(f"Speed Adjustment Latency: {(end_time - start_time) * 1000:.2f} ms")

Speed Adjustment Latency: 100.96 ms
